### Задача №17

Первая камера находится в начале координат, ее оптическая ось
направлена вдоль оси z. Вторая камера сдвинута относительно первой на 1 в
направлении оси x. Матрицы внутренних параметров обеих камер единичные.
Сгенерировать случайный набор точек в трехмерном пространстве с z>0,
спроектировать их на обе камеры (cv2.projectPoints), по проекциям найти
необходимую матрицу (cv2.findEssentialMat)

In [1]:
import numpy as np
import math
import cv2

In [2]:
K1 = np.matrix([[1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]], dtype=np.float64)

K2 = np.matrix([[1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]], dtype=np.float64)
                
R1 = np.matrix([[1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]], dtype=np.float64)
                
R2 = np.matrix([[1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]], dtype=np.float64)

T1 = np.array([0, 0, 0], dtype=np.float64)
T2 = np.array([-1, 0, 0], dtype=np.float64)

In [3]:
points_3d = np.random.uniform(0, 100, size=(1000, 3))
project_points_1, _ = cv2.projectPoints(points_3d, R1, T1, K1, np.array([0, 0, 0, 0],dtype=np.float64))
project_points_2, _ = cv2.projectPoints(points_3d, R2, T2, K2, np.array([0, 0, 0, 0],dtype=np.float64))


In [4]:
E, _ = cv2.findEssentialMat(project_points_1, project_points_2, K1)
print(f"Essential matrix: \n{E}")

Essential matrix: 
[[ 1.41541951e-13  5.11440136e-12  2.76841593e-11]
 [-5.05159905e-12  1.05520265e-13 -7.07106781e-01]
 [-2.75892411e-11  7.07106781e-01  7.92562211e-14]]


### Задача №18

По необходимой матрице найти все возможные позы первой камеры
относительно второй, для этого использовать cv2.decomposeEssentialMat. Из
получившихся 4 решений выбрать одно, используя условие, что трехмерные точки
(сгенерированные в задаче 17) должны находиться в области видимости камер

In [5]:
r1, r2, t = cv2.decomposeEssentialMat(E)
r1, r2, t

(array([[ 1.00000000e+00, -1.34234615e-13,  8.88158748e-14],
        [ 1.32532714e-13,  1.00000000e+00,  1.30673250e-13],
        [-8.82780889e-14, -1.30645495e-13,  1.00000000e+00]]),
 array([[ 1.00000000e+00,  7.81683924e-11, -1.43768957e-11],
        [ 7.81666905e-11, -1.00000000e+00, -1.30673250e-13],
        [-1.43763579e-11,  1.30645494e-13, -1.00000000e+00]]),
 array([[ 1.00000000e+00],
        [ 3.91496116e-11],
        [-7.23231798e-12]]))

In [6]:
P1 = np.concatenate((K1, np.zeros((3,1))), axis=1)
for rotation, translation in [(r1, t), (r1, -t), (r2, t), (r2, -t)]:
    P2 = np.dot(K2, np.concatenate((rotation, translation), axis=1))
    obj_pt = cv2.triangulatePoints(P1, P2, project_points_1, project_points_2)
    obj_pt /= obj_pt[3]
    if (obj_pt >= 0).all():
        print(f"True R: \n{rotation}\n")
        print(f"True T: \n{translation.T}")
        break

True R: 
[[ 1.00000000e+00 -1.34234615e-13  8.88158748e-14]
 [ 1.32532714e-13  1.00000000e+00  1.30673250e-13]
 [-8.82780889e-14 -1.30645495e-13  1.00000000e+00]]

True T: 
[[-1.00000000e+00 -3.91496116e-11  7.23231798e-12]]


### Задача №19

Найти матрицу внутренних параметров камеры изображения
https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing, предполагая, что оптическая ось проходит строго через центр изображения, а
фокусное расстояние в эквиваленте 35мм равно 0.64 дюйма. Почему результат
отличается от параметров, полученных методом калибрации камеры
https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing? 

In [16]:
def camera_matrix(focal_length_inch, image_width, image_height):
    focal_length_mm = focal_length_inch * 25.4 
    fx = image_width * focal_length_mm/ 36
    fy = image_height * focal_length_mm/ 24 
    cx = image_width / 2
    cy = image_height / 2

    K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0, 0, 1]])
    return K

img = cv2.cvtColor(cv2.imread("./Resourses/image.jpg"), cv2.COLOR_RGB2BGR)

image_height = img.shape[0]
image_width = img.shape[1]
focal_length_inch = 0.64

K = camera_matrix(focal_length_inch, image_width, image_height)
print(f"Матрица внутренних параметров камеры:\n{K}")

Матрица внутренних параметров камеры:
[[433.49333333   0.         480.        ]
 [  0.         365.76       270.        ]
 [  0.           0.           1.        ]]


In [13]:
import xml.etree.ElementTree as ET

xml_file = ET.parse(f'./Resourses/camera.xml')
root = xml_file.getroot()

xml_iter = root.iter("data")
camera_matrix = np.array(next(xml_iter).text.split(), np.float32).reshape(3, 3)
distortion_coefficients = np.array(next(xml_iter).text.split(), np.float32).reshape(14)

print(f"Матрица камеры:\n{camera_matrix}")

Матрица камеры:
[[425.81152   0.      481.75803]
 [  0.      435.33255 267.43704]
 [  0.        0.        1.     ]]


Различия возможны из-за того, что метод каллибровки учитывает искажения линзы. Плюс реальный центр изображеняи может отличаться от высчитанного теоретически.